## Zadanie 14.1.
#### Napisz program ‘parking2.py’ różniący się od programu ‘parking.py’ tym, że klienci parkingu mogą posiadać miesięczne karty abonentowi (dodać funkcję sprzedaży). Wjazdy i wyjazdy samochodów takich klientów są rejestrowane, jednak przy wyjeździe z parkingu opłata nie jest pobierana, o ile nie abonament nie skończył się (wyświetla się tylko informacja o liczbie pozostałych dni). Jeżeli abonament danego auta skończył się, przy jego najbliższym wjeździe lub wyjeździe użytkownik jest o tym informowany i może wykupić nowy abonament lub z niego zrezygnować (i w konsekwencji wnieść standardową opłatę). 

In [48]:
import shelve, sys 
from time import *
from datetime import datetime

In [49]:
#inicjalizacja bazy danych 
def init(): 
    global baza, parking, abonamenty, stawka, okres 
    try: 
        baza = shelve.open ('baza_parkingowa', writeback = True) # otwarcie    
    except: 
        print("Błąd krytyczny! Baza danych nie została otwarta!")
        sys.exit(0) # wyjście z programu 
    print("Inicjalizacja udana. Baza danych została otwarta.")
    if '_stawka' in baza.keys(): # czy już istnieje? 
        (stawka,okres)=baza['_stawka'] # tak - kopiujemy stawki 
    else: 
        (stawka,okres)=(0.0,1) # nie - 
        zmiana_stawki() # wczytujemy od użytkownika 
    if 'parking' not in baza.keys():
        baza['parking'] = {}
    parking = baza['parking']
    if 'abonamenty' not in baza.keys():
        baza['abonamenty'] = {}
    abonamenty = baza['abonamenty']       
    aktualizuj_abonamenty()

In [50]:
# menu główne programu 
def menu(): 
    while True: 
        print ()
        print ('-'*70) 
        print ('PARKING'.center(70)) 
        print ('-'*70 )
        print ('[W] Wjazd [E] Wyjazd [P] Pojazdy [S] Stawka [A] Abonament [K] Koniec'.center(70))
        print ('-'*70) 
        w=input()[0].upper() # pierwszy znak (duza litera) 
        if w in 'WEPSAK': # znany? 
            return w # tak - zwracamy go 
        print ('Nieznane polecenie -') 

In [51]:
# realizacja wyboru użytkownika 
def wybor(): 
    while True: 
        w=menu() 
        if w=='K': 
            break 
        elif w=='S': 
            zmiana_stawki() 
        elif w=='P': 
            pojazdy() 
        elif w=='E': 
            wyjazd() 
        elif w=='W': 
            wjazd() 
        elif w=='A':
            abonament()

In [52]:
# zmiana opłat 
def zmiana_stawki(): 
    global baza,stawka,okres # zmienne globalne 
    print("\nZmiana wysokości opłat\n")
    print("Bieżąca stawka wynosi %.2f zł za %i minut(y)\n" % (stawka,okres))
    try: 
        s=float(input("Podaj nową wysokość opłat: ")) 
        o=int(input("Podaj nowy czas naliczania w minutach: ")) 
    except: 
        print("Błąd wprowadzania danych! Stawka nie została zmieniona!") 
        return 
    try: 
        baza['_stawka']=(s,o) # zapisujemy w bazie 
    except: 
        print("Błąd zapisu danych! Stawka nie została zmieniona!") 
    else: 
        stawka=s # kopiujemy do 
        okres=o # zmiennych globalnych 

In [53]:
# lista pojazdów na parkingu 
def pojazdy(): 
    global parking 
    print ()
    print ('Lista pojazdów na parkingu'.center(33)) 
    print ('-'*33) 
    print ('|'+'Nr rej.'.center(10)+'|'+'Godz. parkowania'.center(20)+'|' )
    print( '-'*33 )
    for rej,godz in parking.items(): 
        if rej!='_stawka': 
            print ("|%9s |" % rej,strftime("%H:%M (%Y-%m-%d)",godz),'|' )
    print( '-'*33) 

In [54]:
# rejestracja wjazdu pojazdu 
def wjazd(): 
    global parking, abonamenty
    godz=localtime() 
    print ('Wjazd pojazdu - godzina',strftime("%H:%M (%Y-%m-%d)",godz)) 
    rej=input('Podaj numer rejestracyjny pojazdu: ')[:9] 
    if rej=='_stawka': 
        return # zabezpieczenie 
    if rej not in parking.keys(): # nie jest zaparkowany? 
        parking[rej]=godz 
        print ("Wprowadzono." )
    else: 
        print ("Błąd! Taki pojazd już jest na parkingu!") 

In [59]:
# rejestracja wyjazdu pojazdu 
def wyjazd(): 
    global parking, abonament, stawka, okres 
    print ('Wyjazd pojazdu - godzina',strftime("%H:%M (%Y-%m-%d)"))
    rej=input('Podaj numer rejestracyjny pojazdu: ') 
    if rej in parking.keys(): # czy taki był zaparkowany?
        godz=parking[rej]
        print ("Godzina wjazdu:",strftime("%H:%M (%Y-%m-%d)",godz)) 
        if rej in abonamenty.keys():
            pozostale_dni = 30 - (datetime.now() - abonamenty[rej][0]).days
            if pozostale_dni: 
                print("Pozostało",pozostale_dni,"dni abonamentu. Dziękujemy.")
            else:
                nowy_abonament = input('Twój abonament się skończył. Czy chcesz wykupić nowy? (T/N): ')
                del abonamenty[rej]
                if nowy_abonament == 'T':
                    abonament()
                elif nowy_abonament == 'N':
                    print("Zrezygnowano z abonamentu. Naliczona zostanie standardowa opłata.")
                    minuty=int(mktime(localtime())-mktime(godz))/60 
                    jednostki=(minuty+okres-1)/okres # naliczamy za rozpoczeta 
                    print ("\nDo zapłaty: %.2f zł" % (jednostki*stawka)) 
        else:
            minuty=int(mktime(localtime())-mktime(godz))/60 
            jednostki=(minuty+okres-1)/okres # naliczamy za rozpoczeta 
            print ("\nDo zapłaty: %.2f zł" % (jednostki*stawka)) 
        del parking[rej] # usuwamy wpis 
    else: 
        print ("Błąd! Takiego pojazdu nie ma na parkingu!" )

In [56]:
# wykupienie abonamentu miesięcznego
def abonament():
    global abonamenty
    print ('Zakup abonamentu miesięcznego - godzina',strftime("%H:%M (%Y-%m-%d)"))
    rej=input('Podaj numer rejestracyjny pojazdu: ') 
    data = datetime.now()
    if rej not in abonamenty.keys():
        abonamenty[rej] = [data, 1] # drugi argument wskazuje na to, czy abonament jest aktywny
        print ("Wykupiono abonament." )
    else: 
        print ("Błąd! Ten pojazd już posiada abonament!") 

In [57]:
# aktualizacja abonamentów
def aktualizuj_abonamenty():
    global abonamenty
    now = datetime.now()
    for abonament in abonamenty.items():
        if (now-abonament[0]).days > 30:
            abonament[1] = 0
            print("Abonament się skończył!")

In [58]:
# program główny 
init() # otwarcie bazy 
try: 
     wybor() # interfejs użytkownika 
except: 
     print ("Wystąpił poważny błąd." )
baza.close() # zamknięcie bazy 

Inicjalizacja udana. Baza danych została otwarta.

----------------------------------------------------------------------
                               PARKING                                
----------------------------------------------------------------------
 [W] Wjazd [E] Wyjazd [P] Pojazdy [S] Stawka [A] Abonament [K] Koniec 
----------------------------------------------------------------------
A
Zakup abonamentu miesięcznego - godzina 17:47 (2019-01-14)
Podaj numer rejestracyjny pojazdu: KWAG717
Wykupiono abonament.

----------------------------------------------------------------------
                               PARKING                                
----------------------------------------------------------------------
 [W] Wjazd [E] Wyjazd [P] Pojazdy [S] Stawka [A] Abonament [K] Koniec 
----------------------------------------------------------------------
W
Wjazd pojazdu - godzina 17:47 (2019-01-14)
Podaj numer rejestracyjny pojazdu: KWAG717
Wprowadzono.

-----------